# Dependencies

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

# LLm loading
- We are going to use a Quantized Version of the Gemma2-9B for the resume task
  - Using GenLLm produces a better context understanding than summarization based model, therefore a better quality summarization


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4
load_in_4bit = True # Use 4bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-9b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.6: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


<a name="Inference"></a>
# Inference
- Let's summarize the the relevant casestudies identified in the previous notebook


In [21]:
import pandas as pd

articles = pd.read_csv("/content/Relevant_caseStudies.csv")
articles['Summary']=None
display(articles.head()
)
import torch
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

tokenizer.eos_token = tokenizer.eos_token + tokenizer.pad_token

SUMMARIZATION_SIZE = 256
for index, row in articles.iterrows():

  prompt_text = alpaca_prompt.format(
          f"You are a Summurization expert. The expert should to generate crief and concise {SUMMARIZATION_SIZE} words summarization about the case studies provided and relevant to the 'Decarbonization of Industrial Heat'. ", # instruction
          row['Cleaned_Content']+tokenizer.eos_token + tokenizer.pad_token, # input
          "", # output - leave this blank for generation!
      )
    # Tokenize the prompt
  tokens = tokenizer(prompt_text)

  # Get the number of tokens
  token_size = len(tokens['input_ids'])

  print(f"Token size: {token_size}")

  inputs = tokenizer(
  [
      prompt_text
  ], return_tensors = "pt").to("cuda")

  from transformers import TextStreamer
  text_streamer = TextStreamer(tokenizer)
  with torch.no_grad():
    gen = model.generate(
        **inputs,
        max_new_tokens=SUMMARIZATION_SIZE + 50,  # Allow slightly more tokens than the desired summary length
        min_new_tokens=SUMMARIZATION_SIZE - 10,  # Ensure it doesn't stop too early
        no_repeat_ngram_size=3,  # Avoid repetition of words or phrases
        do_sample=True,  # Enable sampling for more natural text
        temperature=0.4,  # Adjusted to be more deterministic and slighltly creative (To avoid repetition)
        streamer = text_streamer
    )
  # response = tokenizer.batch_decode(gen, skip_special_tokens=True)[0]
  response = tokenizer.batch_decode(gen, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0].strip()

  response = response.split("### Response:")[-1].strip()  # Extract the response

  articles.loc[index, 'Summary'] = response


,Title,Link,Cleaned_Content,Summary
0,Flexible Efficient Electrification of Industri...,https://ispt.eu/projects/flexible-efficient-el...,Flexible Efficient Electrification Industrial ...,None
1,FUSE – FUll ScalE Industrial Heat Pump Using N...,https://ispt.eu/projects/fuse/,FUSE – FUll ScalE Industrial Heat Pump Using N...,None
2,COMTA – COMpact modular Thermo Acoustic heat pump,https://ispt.eu/projects/comta/,COMTA – COMpact modular Thermo Acoustic heat p...,None
3,FLEXSTEAM – Development of heat storage for in...,https://ispt.eu/projects/steam-storing/,FLEXSTEAM – Development heat storage industria...,None
4,SPOT: Sustainable PrOcess heaTing,https://ispt.eu/projects/spot-sustainable-proc...,SPOT: Sustainable PrOcess heaTing UH-30-08 Jan...,None


Token size: 710
<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a Summurization expert. The expert should to generate crief and concise 256 words summarization about the case studies provided and relevant to the 'Decarbonization of Industrial Heat'. 

### Input:
Flexible Efficient Electrification Industrial Heat Cases HP-50-02 Start date November 13, 2023 Electrification heat-driven process needed substantial reduction CO2 emissions. However, challenge efficiently move full electrification coping high variability availability price electricity. Additionally, co-existence natural gas boiler grid connection limitations. project develop potential route based available technology two virtual public simulation cases. simulationas enable partner share promote expertise without disclose confidential information. short: project develop innovative process 

# Cleaning Generated Summaries

In [33]:
# Clean Incomplete summarization
for index, row in articles.iterrows():
  # Removing the last incomlete sentence
  if row['Summary'].split('.')[-1].strip() != '':
    articles.loc[index, 'Summary'] = '.'.join(row['Summary'].split('.')[:-1])

  # Removing repeated generation
  if '##'in row['Summary']:
    articles.loc[index, 'Summary'] = row['Summary'].split('##')[0]


In [34]:
articles

,Title,Link,Cleaned_Content,Summary
0,Flexible Efficient Electrification of Industri...,https://ispt.eu/projects/flexible-efficient-el...,Flexible Efficient Electrification Industrial ...,The project aims to develop an innovative proc...
1,FUSE – FUll ScalE Industrial Heat Pump Using N...,https://ispt.eu/projects/fuse/,FUSE – FUll ScalE Industrial Heat Pump Using N...,The FUSE (FUll Scaled Industrial Heat Pumps Us...
2,COMTA – COMpact modular Thermo Acoustic heat pump,https://ispt.eu/projects/comta/,COMTA – COMpact modular Thermo Acoustic heat p...,The COMTA (COMpact modular thermoacoustic pump...
3,FLEXSTEAM – Development of heat storage for in...,https://ispt.eu/projects/steam-storing/,FLEXSTEAM – Development heat storage industria...,The FLEXSTEAME project is a joint exercise bet...
4,SPOT: Sustainable PrOcess heaTing,https://ispt.eu/projects/spot-sustainable-proc...,SPOT: Sustainable PrOcess heaTing UH-30-08 Jan...,The SPOT project aims to significantly reduce ...
5,The Heat Is On,https://ispt.eu/projects/the-heat-is-on/,"Heat HP-50-01 September 1, 2021 Heat aim make ...",The project aims to reduce CO₂ emissions throu...
6,COMPRESORP,https://ispt.eu/projects/compresorp/,"COMPRESORP UH-20-10 Completed April 1, 2015 Up...",The COMPRESSORP project aims to upgrade low- t...
7,Steam and condensate quality,https://ispt.eu/projects/condensate-quality/,"Steam condensate quality WP-20-11 January 1, 2...",The project aims to improve the insight into t...
8,ENCORE – nExt geNeration COmpRession hEat pump,https://ispt.eu/projects/encore/,ENCORE – nExt geNeration COmpRession hEat pump...,The ENCORE – Next Generation Compression Heat ...
9,LESSON – Oil free compressor for ammonia based...,https://ispt.eu/projects/lesson/,LESSON – Oil free compressor ammonia based hig...,The project aims to improve the performance of...


In [35]:
articles.to_csv('Summarized_relevant_casestudies.csv', index=False)